<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_textsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Text Summarization**
Refs:
- https://huggingface.co/models
- https://huggingface.co/models?search=bertabs
- [BertForMaskedLM](https://github.com/huggingface/transformers/blob/13aa174112f0c2ee794c44188ecf13b241694db0/src/transformers/modeling_bert.py#L876) used as architecture in BertAbs model, which can do
  * **masked language modeling** loss, for language modeling, or
  * left-to-right language modeling loss (**next word prediction**)

Hyper-params:
TBA

Issues:
1. Max sentence length: 64 (truncated)

## Environment Setup
To get the source code of Huggingface transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [21]:
!uname -a
!pip install wget
!apt-get install file

Linux 153b72610239 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
file is already the newest version (1:5.32-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [0]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

In [23]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


## Configuratiuon

In [0]:
EXAMPLE_MODE=True
INLINE_MODE=True

In [25]:
if EXAMPLE_MODE or INLINE_MODE:
  # ! [ ! -d "./transformers" ] && git clone --depth 1 https://github.com/huggingface/transformers
  # ! [ -d "./transformers" ] && \
  #   cd transformers && \
  #   git fetch -a --tags && \
  #   git checkout tags/v2.11.0 && \
  #   pip install . && \
  !rm -rf ./nlp-qa/
  ![ ! -d "./nlp-qa" ] && git clone --depth 1 https://github.com/hailusong/nlp-qa.git
  !pip install nltk py-rouge

Cloning into 'nlp-qa'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 45 (delta 3), reused 28 (delta 2), pack-reused 0
Unpacking objects: 100% (45/45), done.


### GPU Setup

In [26]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


## Download

### Pre-trained BERTABS fine-tuned on CNN/DM corpus 


In [0]:
# if INLINE_MODE:
#   # from transformers import AutoTokenizer, AutoModelForQuestionAnswering
#   # from transformers import AlbertConfig, AlbertForSequenceClassification
#   # import torch
#   from modeling_bertabs import BertAbs, build_predictor
#   from transformers import BertTokenizer

#   tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
#   model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
#   model.to(args.device)
#   model.eval()

### Text to be summarized

In [28]:
if EXAMPLE_MODE:
  %env DATA_PATH=/content/nlp-qa/stories
  %env SUMMARIES_PATH=/content/textsum-out

  !mkdir -p $SUMMARIES_PATH
  !ls -al $SUMMARIES_PATH

if INLINE_MODE:
  DATA_PATH='/content/nlp-qa/stories'
  SUMMARIES_PATH='/content/textsum-out'

  !mkdir -p $SUMMARIES_PATH
  !ls -al $SUMMARIES_PATH


env: DATA_PATH=/content/nlp-qa/stories
env: SUMMARIES_PATH=/content/textsum-out
total 8
drwxr-xr-x 2 root root 4096 Jun 10 18:46 .
drwxr-xr-x 1 root root 4096 Jun 10 18:55 ..
-rw-r--r-- 1 root root    0 Jun 10 18:53 bart_cnn_test_summaries.txt
total 8
drwxr-xr-x 2 root root 4096 Jun 10 18:46 .
drwxr-xr-x 1 root root 4096 Jun 10 18:55 ..
-rw-r--r-- 1 root root    0 Jun 10 18:53 bart_cnn_test_summaries.txt


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


### BertAbs

In [29]:
if EXAMPLE_MODE:
  # !pwd
  # !ls -al transformers/examples/summarization/bertabs
  # !cd transformers/examples/summarization/bertabs && python run_summarization.py \
  #   --documents_dir $DATA_PATH \
  #   --summaries_output_dir $SUMMARIES_PATH \
  #   --no_cuda false \
  #   --batch_size 4 \
  #   --min_length 50 \
  #   --max_length 200 \
  #   --beam_size 5 \
  #   --alpha 0.95 \
  #   --block_trigram true
  # !ls -al nlp-qa/textsum/bertabs
  !cd nlp-qa/textsum/bertabs && python run_summarization.py \
    --documents_dir $DATA_PATH \
    --summaries_output_dir $SUMMARIES_PATH \
    --no_cuda false \
    --batch_size 4 \
    --min_length 20 \
    --max_length 50 \
    --beam_size 5 \
    --alpha 0.95 \
    --block_trigram true

2020-06-10 18:55:53.511620: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
INFO:filelock:Lock 139990310497304 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpuksi2uzg
Downloading: 100% 232k/232k [00:00<00:00, 856kB/s] 
INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/tran

### BART

In [30]:
if EXAMPLE_MODE:
  !ls -al nlp-qa/textsum/bart
  !cd nlp-qa/textsum/bart && python evaluate_cnn.py $DATA_PATH/mine1.story $SUMMARIES_PATH/bart_cnn_test_summaries.txt facebook/bart-large-cnn

total 44
drwxr-xr-x 2 root root 4096 Jun 10 18:55 .
drwxr-xr-x 5 root root 4096 Jun 10 18:55 ..
-rw-r--r-- 1 root root 2553 Jun 10 18:55 evaluate_cnn.py
-rw-r--r-- 1 root root 7175 Jun 10 18:55 finetune.py
-rw-r--r-- 1 root root    0 Jun 10 18:55 __init__.py
-rw-r--r-- 1 root root 1948 Jun 10 18:55 README.md
-rwxr-xr-x 1 root root  488 Jun 10 18:55 run_train.sh
-rwxr-xr-x 1 root root  875 Jun 10 18:55 run_train_tiny.sh
-rw-r--r-- 1 root root 5449 Jun 10 18:55 test_bart_examples.py
-rw-r--r-- 1 root root 2229 Jun 10 18:55 utils.py
2020-06-10 18:59:05.608591: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
100% 1/1 [00:02<00:00,  2.72s/it]


In [31]:
if EXAMPLE_MODE:
  !ls -al $SUMMARIES_PATH
  !wc -l $SUMMARIES_PATH/*
  !grep "" $SUMMARIES_PATH/*

total 44
drwxr-xr-x 2 root root 4096 Jun 10 18:59 .
drwxr-xr-x 1 root root 4096 Jun 10 18:55 ..
-rw-r--r-- 1 root root  683 Jun 10 18:59 bart_cnn_test_summaries.txt
-rw-r--r-- 1 root root  206 Jun 10 18:59 e117408ad19cc69e15b1e21b9ae54f10c07223ce_summary.story
-rw-r--r-- 1 root root  252 Jun 10 18:58 e3dd06d326c8d53722bdc5c8428e29c68a60d0d5_summary.story
-rw-r--r-- 1 root root  228 Jun 10 18:58 e9093ca0a82f2aba28cd2762942c04177cfbb000_summary.story
-rw-r--r-- 1 root root  198 Jun 10 18:58 ea06fd0b25cb9793397a51de73fd83f91b4323fa_summary.story
-rw-r--r-- 1 root root  243 Jun 10 18:59 eb68bc51ed4fc727a1af058192a3fef0916c91e7_summary.story
-rw-r--r-- 1 root root  267 Jun 10 18:58 ee0ba7928d8987f2cb21c9a2012a76730f77de45_summary.story
-rw-r--r-- 1 root root  218 Jun 10 18:59 ee8871b15c50d0db17b0179a6d2beab35065f1e9_summary.story
-rw-r--r-- 1 root root  228 Jun 10 18:59 mine1_summary.story
   2 /content/textsum-out/bart_cnn_test_summaries.txt
   0 /content/textsum-out/e117408ad19cc69e15b1e2

In [0]:
if INLINE_MODE:
  from pathlib import Path
  from collections import namedtuple

  import torch
  from tqdm import tqdm

  from transformers import BartForConditionalGeneration, BartTokenizer


  DEFAULT_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


  def chunks(lst, n):
      """Yield successive n-sized chunks from lst."""
      for i in range(0, len(lst), n):
          yield lst[i : i + n]


  def generate_summaries(
      examples: list, out_file: str, model_name: str, batch_size: int = 8, device: str = DEFAULT_DEVICE,
      max_length = 140, min_length = 55
  ):
      fout = Path(out_file).open("w")
      model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
      tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

      for batch in tqdm(list(chunks(examples, batch_size))):
          dct = tokenizer.batch_encode_plus(batch, max_length=1024, return_tensors="pt", pad_to_max_length=True)
          summaries = model.generate(
              input_ids=dct["input_ids"].to(device),
              attention_mask=dct["attention_mask"].to(device),
              num_beams=4,
              length_penalty=2.0,
              max_length=max_length + 2,  # +2 from original because we start at step=1 and stop before max_length
              min_length=min_length + 1,  # +1 from original because we start at step=1
              no_repeat_ngram_size=3,
              early_stopping=True,
              decoder_start_token_id=model.config.eos_token_id,
          )
          dec = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
          for hypothesis in dec:
              # fout.write(hypothesis + "\n")
              # fout.flush()
              print("\n" + hypothesis)

In [33]:
if INLINE_MODE:
    parser = namedtuple('parser', 'source_path output_path model_name device bs')
    args = parser(
        source_path = f'{DATA_PATH}/mine1.story',
        output_path = f'{SUMMARIES_PATH}/bart_cnn_test_summaries.txt',
        model_name = 'facebook/bart-large-cnn',
        device = DEFAULT_DEVICE,
        bs = 8)

    examples = [" " + x.rstrip() for x in open(args.source_path).readlines()]
    examples = [ " ".join(examples) ]
    print(f"total lines: {len(examples)}, line data: {examples}")
    generate_summaries(examples, args.output_path, args.model_name, batch_size=args.bs, device=args.device, max_length=50, min_length=20)


total lines: 1, line data: [' Here is our updated policy at Clear Living Clinic to keep you safe and provide peace-of-mind. We will be limiting the number of clients in the clinic at all times. We will be allowing only one client at a time in the waiting area. Non-clients will be asked to kindly wait outside the clinic if someone is accompanying you to your appointment. Upon booking an appointment, we will pre-screen every client before they are permitted to visit the clinic. If you have traveled recently, feel under the weather or are taking care of someone who is ill, please call your family doctor or Telehealth Ontario and self-isolate for 14 days.  Check-in: Please wash your hands upon arrival. As usual, we provide you with hand sanitizer and paper towels. We will also provide you with disposable gloves, a mask and a disposable toilet seat cover upon request. Our clinic has always been very proactive with disinfecting all surfaces using hospital grade disinfectant before and after 


100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


Clear Living Clinic will be limiting the number of clients in the clinic at all times. We will be allowing only one client at a time in the waiting area. Non-clients will be asked to kindly wait outside the clinic if someone is accompanying
